In [1]:
# Downloaded libraries
import torch
from torch import nn
from torch_geometric.loader import DataLoader


# Local files
from dataset_graphs import NNDataset
from models import Trainer_GCN

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Constants
TRAINING_SPLIT = 0.8

In [3]:
# Hyperparameters
num_epoch = 100
batch_size = 16

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
nndataset = NNDataset(root="../")

size = len(nndataset)
train_num = int(size * TRAINING_SPLIT)
test_num = size - train_num

train_loader = DataLoader(
    dataset=nndataset[:train_num], batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    dataset=nndataset[train_num:], batch_size=test_num, shuffle=True)

print(
    f"Dataset loaded, {train_num} training samples and {test_num} testing samples")


Dataset loaded, 800 training samples and 200 testing samples


In [6]:
data = iter(train_loader).next()
print(data)

DataBatch(design=[16], edge_index=[2, 34144], x=[1319, 504], y_edge=[17072, 1], y_node=[1319, 1], input_mask=[1319, 1], num_nodes=1319, batch=[1319], ptr=[17])


In [7]:
model = Trainer_GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = nn.MSELoss()

In [8]:
# Model Training
model.train()
for epoch in range(num_epoch):
    print(f"Epoch {epoch + 1} / {num_epoch}:")
    for i, data in enumerate(train_loader):
        data.to(device)

        # forward propagation
        out_w, out_b = model(data)
        loss = loss_fn(out_b, data.y_node) + loss_fn(out_w, data.y_edge)

        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print status every n batches
        if i % 10 == 0:
            loss, current = loss.item(), i * batch_size
            print(
                f"Training Loss: {loss:>7f}  [{current:>5d}/{train_num:>5d}]")

Epoch 1 / 100:
Training Loss: 0.903748  [    0/  800]
Training Loss: 0.347704  [  160/  800]
Training Loss: 0.321701  [  320/  800]
Training Loss: 0.260181  [  480/  800]
Training Loss: 0.343222  [  640/  800]
Epoch 2 / 100:
Training Loss: 0.233592  [    0/  800]
Training Loss: 0.399781  [  160/  800]
Training Loss: 0.384323  [  320/  800]
Training Loss: 0.339276  [  480/  800]
Training Loss: 0.253339  [  640/  800]
Epoch 3 / 100:
Training Loss: 0.322542  [    0/  800]
Training Loss: 0.231674  [  160/  800]
Training Loss: 0.296107  [  320/  800]
Training Loss: 0.268927  [  480/  800]
Training Loss: 0.347784  [  640/  800]
Epoch 4 / 100:
Training Loss: 0.405687  [    0/  800]
Training Loss: 0.344848  [  160/  800]
Training Loss: 0.301651  [  320/  800]
Training Loss: 0.299163  [  480/  800]
Training Loss: 0.337260  [  640/  800]
Epoch 5 / 100:
Training Loss: 0.365052  [    0/  800]
Training Loss: 0.278226  [  160/  800]
Training Loss: 0.401495  [  320/  800]
Training Loss: 0.356757  [  

In [9]:
# Model Evaluation
model.eval()
with torch.no_grad():
    data = iter(test_loader).next().to(device)

    # forward propagation
    out_w, out_b = model(data)
    loss = loss_fn(out_b, data.y_node) + loss_fn(out_w, data.y_edge)

    loss = loss.item()
    print(f"Validation Loss: {loss:>7f}")


Validation Loss: 0.244337


In [38]:
data = nndataset[0]
data = data.to(device)

tensor([ 0.,  0.,  2., 14.], device='cuda:0')

In [ ]:
out_w, out_b = model(data)

In [12]:
torch.save(model.state_dict(), "../model/model")
print("Model saved")

Model saved
